## Simulation of the Alpha Arm with Submechanism at Upper Arm

This script defines a submechansim with its root frame being after the shoulder joint. This allows for inverse dynamics starting at the upper arm joint. The base joint and the shoulder joint are free to rotate, and inverse dynamics is attempted on the remaining two joints (elbow and wrist). Hydrodynamics are taken into account. 

Main addition: ```mechanism_ua_to_wrist = submechanism(mechanism_alpha, upper_arm_body)```

### Currently Implemented

Hydro wrenches are transformed into the submechanism's "root" frame and applied in the inverse dynamics equation. However, the upper arm body has more forces acting on it than accounted for (e.g. the other forces from the shoulder joint), and thus inverse dynamics does not give the correct torques for the other joints to behave as expected. 

In [1]:
using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using MeshCat
using MeshCatMechanisms
using MechanismGeometries
using CoordinateTransformations
using GeometryBasics

include("/home/hkolano/onr-dynamics-julia/simulate_with_ext_forces.jl")
println("Libraries imported.")

Libraries imported.


In [2]:
# model definition
# ------------------------------------------------------------------------
#                            MODEL DEFINITION
# ------------------------------------------------------------------------
vis = Visualizer()
render(vis)
# Define paths to the URDF files
src_dir = dirname(pathof(RigidBodyDynamics))
urdf_file = joinpath(src_dir, "..", "..", "..", "..", "..", "onr-dynamics-julia", "URDFs", "alphaArmCombo.urdf")
mechanism_alpha = parse_urdf(urdf_file; gravity = [0.0, 0.0, 0.0])

delete!(vis)

# Create visuals of the URDFs
visuals = URDFVisuals(urdf_file)
mvis_alpha = MechanismVisualizer(mechanism_alpha, URDFVisuals(urdf_file), vis[:alpha])

# Name the joints of the mechanism
# println(bodies(mechanism_alpha))
#  z_prism, roll_joint, pitch_joint, yaw_joint,
base_joint, shoulder_joint, elbow_joint, wrist_joint = joints(mechanism_alpha)
#  virtual_y, virtual_z, virtual_roll, virtual_pitch, 
~, shoulder_body, upper_arm_body, elbow_body, wrist_body = bodies(mechanism_alpha)
# num_virtual_links = 4

mechanism_ua_to_wrist = submechanism(mechanism_alpha, upper_arm_body)
free_elbow_j, free_wrist_j = joints(mechanism_ua_to_wrist)
free_ua, free_elbow, free_wrist = bodies(mechanism_ua_to_wrist)

shoulder_frame = default_frame(shoulder_body)
upper_arm_frame = default_frame(upper_arm_body)
elbow_frame = default_frame(elbow_body)
wrist_frame = default_frame(wrist_body)
base_frame = root_frame(mechanism_alpha)
# setelement!(mvis_alpha, shoulder_frame)
# # last_link = bodies(mechanism_alpha)[end]
# # body_frame = default_frame(last_link)

frame_names_cob = ["shoulder_cob", "ua_cob", "elbow_cob", "wrist_cob"]
frame_names_com = ["shoulder_com", "ua_com", "elbow_com", "wrist_com"]
cob_vecs = [SVector{3, Float64}([-0.001, -0.003, .032]), SVector{3, Float64}([0.073, 0.0, -0.002]), SVector{3, Float64}([0.015, -0.012, -.003]), SVector{3, Float64}([0.0, 0.0, -.098])]
com_vecs = [SVector{3, Float64}([0.005, -.001, 0.016]), SVector{3, Float64}([0.073, 0.0, 0.0]), SVector{3, Float64}([0.017, -0.026, -0.002]), SVector{3, Float64}([0.0, 0.003, -.098])]
cob_frames = []
com_frames = []
for i in 1:4
    bod = bodies(mechanism_alpha)[i+1]
    frame_cob = CartesianFrame3D(frame_names_cob[i])
    frame_com = CartesianFrame3D(frame_names_com[i])
    cob_vec = cob_vecs[i]
    com_vec = com_vecs[i]
    cob_transform = Transform3D(frame_cob, default_frame(bod), cob_vec)
    com_transform = Transform3D(frame_com, default_frame(bod), com_vec)
    # setelement!(mvis_alpha, default_frame(bod))
    if !(is_fixed_to_body(bod, frame_cob))
        add_frame!(bod, cob_transform)
        push!(cob_frames, frame_cob)
        # setelement!(mvis_alpha, frame_cob)    # visualizes COB frames in MeshCat
    end
    if !(is_fixed_to_body(bod, frame_com))
        add_frame!(bod, com_transform)
        push!(com_frames, frame_com)
        # setelement!(mvis_alpha, frame_com)    # visualizes COM frames in MeshCat
    end
end

# alphabase_com_wrt_linkframe = SVector{3, Float64}([-0.075, -0.006, -.003])
# alphabase_com_wrt_linkframe = SVector{3, Float64}([-0.075, -0.0, -.0])
# linkframe_wrt_vehframe = translation(frame_definitions(vehicle_body)[38])
# # IF THE ARM IS ROTATED THIS HAS TO CHANGE!!!!
# alphabase_com_wrt_vehframe = alphabase_com_wrt_linkframe + linkframe_wrt_vehframe
# alphabase_com_frame = CartesianFrame3D("armbase_com_cob")
# com_transform = Transform3D(alphabase_com_frame, default_frame(vehicle_body), alphabase_com_wrt_vehframe)

# if !(is_fixed_to_body(vehicle_body, alphabase_com_frame))
#     add_frame!(vehicle_body, com_transform)
#     push!(cob_frames, alphabase_com_frame)
#     push!(com_frames, alphabase_com_frame)
#     # setelement!(mvis_alpha, alphabase_com_frame)
# end
# print("THIS SHOULD SAY before_r5m_joint5: ")
# println(frame_definitions(vehicle_body)[38].from)
# # println(alphabase_com_wrt_vehframe)

println("URDFs parsed. Visualizers ready. \n")

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/hkolano/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


URDFs parsed. Visualizers ready. 



In [3]:
# buoyancy force setup
# ------------------------------------------------------------------------
#                           BUOYANCY SETUP
# ------------------------------------------------------------------------
# f = rho * g * V
# f = 997 (kg/m^3) * 9.81 (m/s^2) * V_in_L *.001 (m^3) = kg m / s^2
# One time setup of buoyancy forces
volumes = [.018, .203, .025, .155] #  shoulder, ua, elbow, wrist,
buoy_force_mags = volumes * 997 * 9.81 * .001
buoy_lin_forces = []
for mag in buoy_force_mags
    lin_force = FreeVector3D(base_frame, [0.0, 0.0, mag])
    push!(buoy_lin_forces, lin_force)
end
print(buoy_lin_forces)

masses = [.194, .429, .115, .333]
grav_forces = masses*9.81
grav_lin_forces = []
for f_g in grav_forces
    lin_force = FreeVector3D(base_frame, [0.0, 0.0, -f_g])
    push!(grav_lin_forces, lin_force)
end
# println(spatial_inertia(vehicle_body))
# println(spatial_inertia(shoulder_body))
# println(spatial_inertia(upper_arm_body))
# println(spatial_inertia(elbow_body))
# println(spatial_inertia(wrist_body))
# println(mass(mechanism_alpha))

Any[

FreeVector3D in "world": [0.0, 0.0, 0.17605026], FreeVector3D in "world": [0.0, 0.0, 1.9854557100000003], FreeVector3D in "world": [0.0, 0.0, 0.24451425000000004], FreeVector3D in "world": [0.0, 0.0, 1.5159883500000002]]

In [4]:
# Hydrodynamics calculator function
# ------------------------------------------------------------------------
#                HYDRODYNAMICS (Grav & Buoy) CALCULATOR
# ------------------------------------------------------------------------
function hydro_calc!(hydro_wrenches::Dict{BodyID, Wrench{Float64}}, t, state_alpha::MechanismState)
    buoy_wrenches = []
    grav_wrenches = []
    names = ["cob1", "cob2", "cob3", "cob4"]
    # Iterate through buoyancy for each body
    for i in 1:4
        # Get the body
        bod = bodies(mechanism_alpha)[i+1]
        # Get default frame of te body
        body_default_frame = default_frame(bod)
        # Get transform between the defualt frame and the center of buoyancy
        # TODO: don't use fixed_transform because it's bad for computation time
        def_to_cob = fixed_transform(bod, body_default_frame, cob_frames[i])
        def_to_com = fixed_transform(bod, body_default_frame, com_frames[i])
        # Transform buoyancy force vector to the body's default frame (rotation only)
        buoy_force_trans = transform(state_alpha, buoy_lin_forces[i], body_default_frame)
        grav_force_trans = transform(state_alpha, grav_lin_forces[i], body_default_frame)
        # Make the wrench: the buoyancy force through a point, the center of buoyancy.
        buoy_wrench = Wrench(Point3D(body_default_frame, translation(inv(def_to_cob))), buoy_force_trans)
        grav_wrench = Wrench(Point3D(body_default_frame, translation(inv(def_to_com))), grav_force_trans)
        # Add wrench to buoy_wrenches
        push!(buoy_wrenches, buoy_wrench)
        push!(grav_wrenches, grav_wrench)
        # Add the buoyancy wrench and grav wrench together
        wrench = buoy_wrench + grav_wrench
        # println(wrench)
        # setelement!(mvis_alpha, Point3D(body_default_frame, translation(inv(def_to_cob))), 0.02, "name")
        # Transform the wrench to the root frame and assign it to the body
        hydro_wrenches[BodyID(bod)] = transform(state_alpha, wrench, root_frame(mechanism_alpha))
    end
end;

sub_wrenches = Dict{BodyID, Wrench{Float64}}()


Dict{BodyID, Wrench{Float64}}()

In [6]:
function get_des_acc!(des_acc, t)
    # TODO: implement a simple pt a to pt b trajectory generation
    # des_acc[base_joint][1] = 0.0
    # des_acc[shoulder_joint][1] = vdot[shoulder_joint][1]
    des_acc[free_elbow_j][1] = 0.0 #vdot[elbow_joint][1]
    des_acc[free_wrist_j][1] = 0.00
end;

In [7]:
function match_states!(state_sub, state_alpha)
    # Set state_sub elbow/wrist vels and pos to be state_alpha vels and pos
    # NOTE: Clears state_sub cache
    set_velocity!(state_sub, free_elbow_j, velocity(state_alpha, elbow_joint))
    set_velocity!(state_sub, free_wrist_j, velocity(state_alpha, wrist_joint))
    set_configuration!(state_sub, free_elbow_j, configuration(state_alpha, elbow_joint))
    set_configuration!(state_sub, free_wrist_j, configuration(state_alpha, wrist_joint))

end;

In [28]:
function match_wrenches!(sub_hwrenches::Dict{BodyID, Wrench{Float64}}, hydro_wrenches, state_alpha)
    sub_hwrenches[BodyID(free_ua)] = transform(state_alpha, hydro_wrenches[BodyID(upper_arm_body)], root_frame(mechanism_ua_to_wrist))
    sub_hwrenches[BodyID(free_elbow)] = transform(state_alpha, hydro_wrenches[BodyID(elbow_body)], root_frame(mechanism_ua_to_wrist))
    # sub_hwrenches[BodyID(free_elbow)] = hydro_wrenches[BodyID(elbow_body)]
    sub_hwrenches[BodyID(free_wrist)] = transform(state_alpha, hydro_wrenches[BodyID(wrist_body)], root_frame(mechanism_ua_to_wrist))
end;

In [43]:
# Controller function
# ------------------------------------------------------------------------
#                              CONTROLLER
# ------------------------------------------------------------------------

function simple_control!(torques::AbstractVector, t, state_alpha::MechanismState, hydro_wrenches::Dict{BodyID, Wrench{Float64}}, vdot)
    # Get buoyancy forces 
    # hydro_wrenches = Dict{BodyID, Wrench{Float64}}()
    # hydro_calc!(hydro_wrenches, t, state_alpha) 
    # Calculate inverse dynamics of alpha arm (including buoyancy)
    get_des_acc!(des_acc, t)
    match_states!(state_sub, state_alpha)
    
    match_wrenches!(sub_wrenches, hydro_wrenches, state_alpha)
    # println(hydro_wrenches)
    # println(sub_wrenches)
    tau = inverse_dynamics(state_sub, des_acc, sub_wrenches)
    # tau = inverse_dynamics(state_sub, des_acc)

    # Set torques accordingly (controller)
    torques[velocity_range(state_alpha, base_joint)] .= 0.0
    torques[velocity_range(state_alpha, shoulder_joint)] .= 0.0 #tau[shoulder_joint][1]
    torques[velocity_range(state_alpha, elbow_joint)] .= tau[free_elbow_j][1]
    torques[velocity_range(state_alpha, wrist_joint)] .= tau[free_wrist_j][1]

end;

In [44]:
# initial conditions
# ------------------------------------------------------------------------
#                          INITIAL CONDITIONS
# ------------------------------------------------------------------------
# Initialize state of alpha arm
state_alpha = MechanismState(mechanism_alpha)
zero!(state_alpha)
set_configuration!(state_alpha, base_joint, 3.141)
# set_configuration!(state, shoulder_joint, -.27079)
# set_configuration!(state, elbow_joint, 3.1415/2+1.3)
# set_configuration!(state, wrist_joint, 0.5)

state_sub = MechanismState(mechanism_ua_to_wrist)
match_states!(state_sub, state_alpha)

# Define desired accelerations
des_acc = similar(velocity(state_sub))
# des_acc[yaw_joint][1] = 0.
# des_acc[base_joint][1] = 0.0
# des_acc[shoulder_joint][1] = 0.0
des_acc[free_elbow_j][1] = 0.0
des_acc[free_wrist_j][1] = 0.0
# print(des_acc)



0.0

In [45]:
# simulation
# ------------------------------------------------------------------------
#                              SIMULATION
# ------------------------------------------------------------------------
final_time = 2
ts, qs, vs = simulate_with_ext_forces(state_alpha, final_time, hydro_calc!, simple_control!; Δt = 5e-3)
# ts, qs, vs = simulate(state_alpha, final_time, simple_control!; Δt = 1e-2)

println("Simulation finished.")

In [46]:
# ------------------------------------------------------------------------
#                             VISUALIZATION
# ------------------------------------------------------------------------
# vis = Visualizer()
# open(vis)

# delete!(vis)
# mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf_file))
# set_configuration!(mvis, [0.0, 0.0])
# open(mvis)
MeshCatMechanisms.animate(mvis_alpha, ts, qs; realtimerate = 0.5)
print(last(qs))

# animation = Animation(mvis, ts, qs)
# setanimation!(mvis, animation)

println("\n done.")